# Postgres Vector Store
In this notebook we are going to show how to use [Postgresql](https://www.postgresql.org) and  [pgvector](https://github.com/pgvector/pgvector)  to perform vector searches in LlamaIndex

In [ ]:
# import logging
# import sys

# Uncomment to see debug logs
# logging.basicConfig(stream=sys.stdout, level=logging.DEBUG)
# logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

from llama_index import SimpleDirectoryReader, StorageContext
from llama_index.indices.vector_store import VectorStoreIndex
from llama_index.vector_stores import PGVectorStore
import textwrap

### Setup OpenAI
The first step is to configure the openai key. It will be used to created embeddings for the documents loaded into the index

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = "<your api key>"

### Loading documents
Load the documents stored in the `paul_graham_essay` using the SimpleDirectoryReader

In [ ]:
documents = SimpleDirectoryReader('../data/paul_graham').load_data()
print('Document ID:', documents[0].doc_id, 'Document Hash:', documents[0].doc_hash)

### Create the index
Here we create an index backed by Postgres using the documents loaded previously. PGVectorStore takes a few arguments.

In [ ]:
conn_str = PGVectorStore.conn_str_from_params(
            database="<db>",
            host="<host>",
            password="<password>",
            port=5432,
            user="<username>")
vector_store = PGVectorStore(
    connection_string=conn_str,
    table_name="paul_graham_essay",
)

storage_context = StorageContext.from_defaults(vector_store=vector_store)
index = VectorStoreIndex.from_documents(documents, storage_context=storage_context)
storage_context.persist(persist_dir="./storage")

### Query the index
We can now ask questions using our index.

In [ ]:
query_engine = index.as_query_engine()
response = query_engine.query("Who is the author?")

In [ ]:
print(textwrap.fill(str(response), 100))

In [ ]:
response = query_engine.query("What did the author do growing up?")

In [ ]:
print(textwrap.fill(str(response), 100))

### Querying existing index

In [ ]:
from llama_index import load_index_from_storage

conn_str = PGVectorStore.conn_str_from_params(
            database="<db>",
            host="<host>",
            password="<password>",
            port=5432,
            user="<username>")
vector_store = PGVectorStore(
    connection_string=conn_str,
    table_name="paul_graham_essay",
)


storage_context = StorageContext.from_defaults(
            vector_store=vector_store,
            persist_dir="./storage"
        )

index = load_index_from_storage(
            storage_context=storage_context,
        )

query_engine = index.as_query_engine()
response = query_engine.query("Who is the author?")